# reading the file


In [188]:
import pandas as pd
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf


In [189]:
df=pd.read_parquet("/content/data.parquet")

In [190]:
df.reset_index(inplace=True)
data = df.copy()
newdf = pd.DataFrame(columns=['Model', 'Absolute P/L','Sharpe Value', 'Max Drawdown'])
min_tf = 30 # 30 minutes
max_tf = 1880 #5 Days

In [191]:
data

,time,banknifty,nifty,tte
0,2021-01-01 09:15:00,0.286058,0.199729,27
1,2021-01-01 09:16:00,0.285381,0.200433,27
2,2021-01-01 09:17:00,0.284233,0.200004,27
3,2021-01-01 09:18:00,0.286104,0.199860,27
4,2021-01-01 09:19:00,0.285539,0.198951,27
...,...,...,...,...
690507,2022-06-30 15:26:00,0.240701,0.214758,28
690508,2022-06-30 15:27:00,0.240875,0.216558,28
690509,2022-06-30 15:28:00,0.242115,0.216794,28
690510,2022-06-30 15:29:00,0.243426,0.216455,28


chnage the dates in between trading hours

In [192]:
data.rename(columns={'time': 'datetime'}, inplace=True)
data['time'] = data['datetime'].astype(str)
data['time'] = pd.to_datetime(data['time'])
data['time'] = data['time'].dt.time
start_trade = pd.to_datetime('09:15:00').time()
end_trade = pd.to_datetime('15:30:00').time()
data = data[(data['time'] >= start_trade) & (data['time'] <= end_trade)]

In [193]:
data['date'] = data['datetime'].astype(str)
data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].dt.date
data['day'] = pd.to_datetime(data['date']).dt.day_name()


<ipython-input-193-e7138d2a9265>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['datetime'].astype(str)
<ipython-input-193-e7138d2a9265>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
<ipython-input-193-e7138d2a9265>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [194]:
data = data[(data['day'] != 'Saturday') & (data['day'] != 'Sunday')]

In [195]:
print("total no. of trading days = ", data['date'].nunique())

total no. of trading days =  345


In [196]:
data[data.isnull().any(axis=1)]

,datetime,banknifty,nifty,tte,time,date,day
204,2021-01-01 12:39:00,0.256850,NaN,27,12:39:00,2021-01-01,Friday
4398,2021-01-04 10:33:00,0.258322,NaN,24,10:33:00,2021-01-04,Monday
4570,2021-01-04 13:25:00,NaN,0.184289,24,13:25:00,2021-01-04,Monday
5969,2021-01-05 12:44:00,0.262732,NaN,23,12:44:00,2021-01-05,Tuesday
7363,2021-01-06 11:58:00,0.262900,NaN,22,11:58:00,2021-01-06,Wednesday
...,...,...,...,...,...,...,...
687514,2022-06-28 13:33:00,NaN,0.223658,2,13:33:00,2022-06-28,Tuesday
687532,2022-06-28 13:51:00,0.235283,NaN,30,13:51:00,2022-06-28,Tuesday
688901,2022-06-29 12:40:00,0.235789,NaN,29,12:40:00,2022-06-29,Wednesday
689022,2022-06-29 14:41:00,0.236893,NaN,29,14:41:00,2022-06-29,Wednesday


In [197]:
data.fillna(method='ffill', inplace=True)


In [198]:
data['Spread'] = data['banknifty'] - data['nifty']
data1=data.copy()
data2=data.copy()

In [199]:
#data['signal'] = 0  # Initialize the column with no signal
# data['signal'] = np.where(data['z_score'] > 1, -1, np.where(data['z_score'] < -1, 1, 0))

In [200]:
# figs=(8,4)
# data['signal'].plot(figsize=figs, linestyle="--")
# data['z_score'].plot(figsize=figs)
# plt.title("Mean Reversion with z-score")
# plt.legend()
# plt.show()

In [201]:
# data['PnL'] = data['Spread'] * (data['tte'] ** 0.7) * data['signal']

In [202]:
# print("Total Profit and loss = ",data['PnL'].sum())

Z score based trading model


In [203]:
data['Rollm'] = data['Spread'].rolling(window=min_tf).mean()
data['Rolls'] = data['Spread'].rolling(window=min_tf).std()
data['z_score'] = zscore(data['Spread'])

In [204]:
data['Long_Signal'] = np.where((data['z_score'] > 1), 1, 0)
data['Short_Signal'] = np.where((data['z_score'] < -1), 1, 0)
train_size=0.7
start_index = int(len(data) * train_size)
data.loc[:start_index, 'Long_Signal'] = 0
data.loc[:start_index, 'Short_Signal'] = 0

In [205]:
data.columns

Index(['datetime', 'banknifty', 'nifty', 'tte', 'time', 'date', 'day',
       'Spread', 'Rollm', 'Rolls', 'z_score', 'Long_Signal', 'Short_Signal'],
      dtype='object')

In [206]:
data['Position_Hold_Time'] = 0
data['Signal Type'] = ''
data['Posi'] = 0
data['Position'] = 0
position_hold_time = 0
current_signal_type = None
position = None

for i in range(len(data)):
    if data['Long_Signal'].iloc[i] == 1:
        current_signal_type = 'long'
    elif data['Short_Signal'].iloc[i] == 1:
        current_signal_type = 'short'
    else:
        current_signal_type = None

    if position_hold_time > max_tf:
        position_hold_time = 0
        current_signal_type = None
        position = None

    if position_hold_time <= min_tf:
        if position_hold_time == 0 and position is None:
            if data["Long_Signal"].iloc[i] == 1:
                position_hold_time = 0
                position = 'long'
            elif data["Short_Signal"].iloc[i] == 1:
                position_hold_time = 0
                position = 'short'
        else:
            position_hold_time += 1
    else:
        if current_signal_type == 'long' and position == 'short':
            position_hold_time = 1
            position = 'long'
        elif current_signal_type == 'short' and position == 'long':
            position_hold_time = 1
            position = 'short'
        elif current_signal_type == position:
            position_hold_time += 1
        else:
            position_hold_time += 1

    data.loc[data.index[i], 'Position_Hold_Time'] = position_hold_time
    data.loc[data.index[i], 'Signal Type'] = current_signal_type
    data.loc[data.index[i], 'Position'] = position

    if data['Position'].iloc[i] == 'long' or data['Position'].iloc[i] == 'short':
        data.loc[data.index[i], 'Posi'] = int(1)

data['PnL'] = data['Spread'] * (data['tte'] ** 0.7) * data['Posi']

data['PnL'] = data['PnL'].fillna(0)
data['Position'] = data['Position'].shift(1)
data['Position_Hold_Time'] = data['Position_Hold_Time'].shift(1)
data['Cumulative_PnL'] = data['PnL'].cumsum()

In [207]:
sharpe = data['PnL'].mean() / data['PnL'].std()
total_pnl = data['PnL'].sum()
max_dd = (data['Cumulative_PnL'].cummax() - data['Cumulative_PnL']).max()
length = len(data)

new_row = {'Model': "Z-score",
           'Absolute P/L': total_pnl,
           'Sharpe Value': sharpe,
           'Max Drawdown': max_dd
           }

newdf = pd.concat([newdf, pd.DataFrame([new_row])], ignore_index=True)

In [208]:
print(newdf)

     Model  Absolute P/L  Sharpe Value  Max Drawdown
0  Z-score  37439.188022      0.830547      0.076773


XG boost based model

In [210]:
from xgboost import XGBRegressor
data['Spread_Lagged_1'] = data['Spread'].shift(1)
data['Spread_Lagged_2'] = data['Spread'].shift(2)

# Fill NaN values in lagged features with 0
data[['Spread_Lagged_1', 'Spread_Lagged_2']] = data[['Spread_Lagged_1', 'Spread_Lagged_2']].fillna(0)

# Create the target variable (y): The future spread
data['Future_Spread'] = data['Spread'].shift(-1)
data[['Future_Spread']] = data[['Future_Spread']].fillna(0)

# Prepare features (X) and target variable (y)
X = data[['Spread_Lagged_1', 'Spread_Lagged_2']]
y = data['Future_Spread']

# Split data into training and testing sets
split_index = int(train_size * len(data)) - min_tf
X_train, X_test, y_train, y_test = X[:split_index], X[split_index:], y[:split_index], y[split_index:]

# Initialize the XGBoost model
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
data['Predicted_Spread'] = 0
data.loc[X_test.index, 'Predicted_Spread'] = xgb_model.predict(X_test)
data['Dynamic_Threshold'] = data['Predicted_Spread'].rolling(window=min_tf).mean().shift(-1)

# Set threshold values for trading signals using the multiplier
threshold = data['Dynamic_Threshold'] * 0.95

# Generate trading signals based on predicted spread in the test set
data['Long_Signal'] = np.where(data['Predicted_Spread'] > threshold, 1, 0)
data['Short_Signal'] = np.where(data['Predicted_Spread'] < -threshold, 1, 0)

# It's assumed that `timeframe` and `evaluation` functions are defined elsewhere
data['Position_Hold_Time'] = 0
data['Signal Type'] = ''
data['Posi'] = 0
data['Position'] = 0
position_hold_time = 0
current_signal_type = None
position = None

for i in range(len(data)):
    if data['Long_Signal'].iloc[i] == 1:
        current_signal_type = 'long'
    elif data['Short_Signal'].iloc[i] == 1:
        current_signal_type = 'short'
    else:
        current_signal_type = None

    if position_hold_time > max_tf:
        position_hold_time = 0
        current_signal_type = None
        position = None

    if position_hold_time <= min_tf:
        if position_hold_time == 0 and position is None:
            if data["Long_Signal"].iloc[i] == 1:
                position_hold_time = 0
                position = 'long'
            elif data["Short_Signal"].iloc[i] == 1:
                position_hold_time = 0
                position = 'short'
        else:
            position_hold_time += 1
    else:
        if current_signal_type == 'long' and position == 'short':
            position_hold_time = 1
            position = 'long'
        elif current_signal_type == 'short' and position == 'long':
            position_hold_time = 1
            position = 'short'
        elif current_signal_type == position:
            position_hold_time += 1
        else:
            position_hold_time += 1

    data.loc[data.index[i], 'Position_Hold_Time'] = position_hold_time
    data.loc[data.index[i], 'Signal Type'] = current_signal_type
    data.loc[data.index[i], 'Position'] = position

    if data['Position'].iloc[i] == 'long' or data['Position'].iloc[i] == 'short':
        data.loc[data.index[i], 'Posi'] = int(1)

data['PnL'] = data['Spread'] * (data['tte'] ** 0.7) * data['Posi']

data['PnL'] = data['PnL'].fillna(0)
data['Position'] = data['Position'].shift(1)
data['Position_Hold_Time'] = data['Position_Hold_Time'].shift(1)
data['Cumulative_PnL'] = data['PnL'].cumsum()
sharpe = data['PnL'].mean() / data['PnL'].std()
total_pnl = data['PnL'].sum()
max_dd = (data['Cumulative_PnL'].cummax() - data['Cumulative_PnL']).max()
length = len(data)

new_row1 = {'Model': "XG Boost",
           'Absolute P/L': total_pnl,
           'Sharpe Value': sharpe,
           'Max Drawdown': max_dd
           }

newdf = pd.concat([newdf, pd.DataFrame([new_row1])], ignore_index=True)

In [212]:
print(newdf)

      Model  Absolute P/L  Sharpe Value  Max Drawdown
0   Z-score  37439.188022      0.830547      0.076773
1  XG Boost  13430.682739      0.505611      0.000000


arima


In [ ]:
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = auto_arima(data1, seasonal=False, stepwise=True, suppress_warnings=True, error_action="ignore", max_order=None, trace=True)